In [26]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


from keras.layers import Input, Dense, Flatten,Activation
from keras.layers import Conv1D, MaxPooling1D,Dropout,LSTM
from keras.layers import Embedding

from numpy import zeros
from numpy import asarray

from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential

from keras.layers import Dense
from keras.layers import Flatten

from keras.layers.embeddings import Embedding

from gensim.models import FastText

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [27]:
new_sentences = list()

training_file = '../data/train.tsv'

for i, line in enumerate(open(training_file).readlines()):
    row = line.strip().split('\t')
    if len(row) > 1:
        tokens = row[1]

        tokens = 'BEGIN' + tokens # + 'END'

        tokens = tokens.replace(' ', ' SPACE ')

        tokens = tokens.replace('>', ' ')

        tokens = tokens.replace('BEGIN', 'BEGIN ')
        # tokens = tokens.replace('END', ' END')

        new_sentences.append(tokens)

In [28]:
new_sentences[:1]

['BEGIN а маравкэва ра тэн SPACE таа ’ко йӈы н']

In [29]:
for i in range(len(new_sentences)):
    new_sentences[i] = new_sentences[i].split(' ')

In [30]:
new_sentences[:1]

[['BEGIN', 'а', 'маравкэва', 'ра', 'тэн', 'SPACE', 'таа', '’ко', 'йӈы', 'н']]

In [31]:
max_len = 0
for sent in new_sentences:
    if len(sent) > max_len:
        max_len = len(sent)

In [32]:
max_len

130

In [38]:
generated_sentences = list()
target_of_generated_sentences = list()
for i, sent in enumerate(new_sentences):
    for j in range(len(sent)-1):
        if j == 0:
            generated_sentences.append(sent[j])
        else:
            generated_sentences.append(sent[:j+1])
        target_of_generated_sentences.append(sent[j+1])


In [39]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(generated_sentences)

vocab_size = len(tokenizer.word_index) + 1
word_index = tokenizer.word_index

In [40]:
encoded_docs = tokenizer.texts_to_sequences(generated_sentences)

In [41]:
encoded_docs[1]

[2, 16]

In [42]:
padded_docs = pad_sequences(encoded_docs, maxlen=max_len, padding='post')

In [51]:
padded_docs[1]

array([ 2, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [43]:
padded_docs

array([[  14,    0,    0, ...,    0,    0,    0],
       [   2,   16,    0, ...,    0,    0,    0],
       [   2,   16, 2532, ...,    0,    0,    0],
       ...,
       [   2,  440,   45, ...,    0,    0,    0],
       [   2,  440,   45, ...,    0,    0,    0],
       [   2,  440,   45, ...,    0,    0,    0]], dtype=int32)

In [44]:
le = LabelEncoder()
target_of_generated_sentences = le.fit_transform(target_of_generated_sentences)

In [52]:
target_of_generated_sentences

array([   5, 5124, 7884, ..., 3074, 6096, 5983])

In [55]:
different_labels = len(pd.unique(target_of_generated_sentences))

In [59]:
target_of_generated_sentences = to_categorical(np.asarray(target_of_generated_sentences))

In [61]:
target_of_generated_sentences.shape

(418546, 17406)

In [62]:
model = Sequential()
model.add(Embedding(vocab_size, 20, input_length=max_len))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(different_labels, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(padded_docs, target_of_generated_sentences, epochs=10, verbose=1)

Epoch 1/10
 80736/418546 [====>.........................] - ETA: 25:02 - loss: 6.1135 - acc: 0.2422

KeyboardInterrupt: 